<img src='http://hilpisch.com/taim_logo.png' width="350px" align="right">

# AI-First Finance

**Deep Learning Improvements**

Dr Yves J Hilpisch | The AI Machine

http://aimachine.io | http://twitter.com/dyjh

## Imports

In [ ]:
!git clone https://github.com/tpq-classes/ai_in_finance.git
import sys
sys.path.append('ai_in_finance')


In [ ]:
import math
import numpy as np
import pandas as pd
from pylab import plt
plt.style.use('seaborn-v0_8')

## Universal Approximation

### With Keras

In [ ]:
x = np.linspace(0, 5 * np.pi, 100)
y = np.cos(x)

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(x, y, 'ro');

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [ ]:
import tensorflow as tf
from keras.layers import Dense
from keras.models import Sequential

In [ ]:
model = Sequential()
model.add(Dense(8 * 48, input_dim=1, activation='relu'))
model.add(Dense(4 * 24, activation='relu'))

In [ ]:
# estimation
model.add(Dense(1, activation='linear'))
model.compile(loss='mse', optimizer='adam', metrics=['mse', 'accuracy'])

In [ ]:
model.summary()

In [ ]:
%time model.fit(x, y, epochs=2000, verbose=False)

In [ ]:
scores = model.evaluate(x, y)

In [ ]:
print('mse: %.5f' % (scores[1]))

In [ ]:
# estimation
pred = model(tf.convert_to_tensor(x, dtype=tf.float32), training=False).numpy()

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(x, y, 'ro', label='original data')
plt.plot(x, pred, label='prediction')
plt.legend();

## Activation Functions

In [ ]:
def activation(x, act='linear', deriv=False):
    if act == 'sigmoid':
        if deriv:
            out = activation(x, 'sigmoid', False)
            return out * (1 - out)
        return 1 / (1 + np.exp(-x))
    elif act == 'relu':
        if deriv:
            return np.where(x > 0, 1, 0)
        return np.maximum(x, 0)
    elif act == 'softplus':
        if deriv:
            return activation(x, act='sigmoid')
        return np.log(1 + np.exp(x))
    elif act == 'linear':
        if deriv:
            return 1
        return x
    else:
        raise ValueError('Activation function not known.')

In [ ]:
x = np.linspace(-1, 1, 20)

In [ ]:
activation(x, 'sigmoid')

In [ ]:
activation(x, 'sigmoid', True)

## Learning &mdash; Simple Neural Network

### Features & Labels

In [ ]:
features = 5
samples = 5

In [ ]:
np.random.seed(10)
l0 = np.random.random((samples, features))
l0  # input layer (features)

In [ ]:
np.linalg.matrix_rank(l0)

In [ ]:
y = np.random.random((samples, 1))   # labels
y

In [ ]:
reg = np.linalg.lstsq(l0, y, rcond=-1)[0]

In [ ]:
reg

In [ ]:
np.allclose(np.dot(l0, reg), y)

### Function

In [ ]:
def snn_function(l0, y, act='linear', a=0.01, steps=100, verbose=False):
    samples, features = l0.shape
    w = np.random.random((features, 1))
    for s in range(1, steps + 1):
        l2 = activation(np.dot(l0, w))
        e = l2 - y
        d = e * activation(l2, act, True)
        u = a * np.dot(l0.T, d)
        w -= u
        mse = (e ** 2).mean()
        if s % 20 == 0 and verbose:
            print(f'step={s:3d} | mse={mse:.5f}')
    return w, mse

### Estimation

In [ ]:
w, mse = snn_function(l0, y, a=0.05, steps=50000, act='linear')
mse

In [ ]:
res = pd.DataFrame({'y': y.flatten(),
                    'pred': np.dot(l0, w).flatten()},
                   index=range(len(y)))

In [ ]:
res.plot(kind='bar', figsize=(10, 6));

### First Class

In [ ]:
class snn:
    def __init__(self, act='linear', a=0.01, steps=100, verbose=False):
        self.act= act
        self.a = a
        self.steps = steps
        self.verbose = verbose
    def fit(self, l0, y, steps=None):
        if steps is None:
            steps = self.steps
        samples, features = l0.shape
        w = np.random.random((features, 1))
        for s in range(1, steps + 1):
            l2 = activation(np.dot(l0, w), self.act)
            e = l2 - y
            d = e * activation(l2, self.act, True)
            u = self.a * np.dot(l0.T, d)
            w -= u
            mse = (e ** 2).mean()
            if s % 20 == 0 and self.verbose:
                print(f'step={s:3d} | mse={mse:.5f}')
        self.w = w
        self.mse = mse
        print(f'mse={mse:.5f}')
    def predict(self, l0):
        return activation(np.dot(l0, self.w), self.act)

### Estimation

In [ ]:
model = snn(a=0.0025, act='softplus', steps=200000)

In [ ]:
model.fit(l0, y)

In [ ]:
model.predict(l0)

In [ ]:
model.predict(l0) - y

In [ ]:
res = pd.DataFrame({'y': y.flatten(),
                    'pred': model.predict(l0).flatten()},
                   index=range(len(y)))

In [ ]:
res.plot(kind='bar', figsize=(10, 6));

### Classification

In [ ]:
features = 5
samples = 10

In [ ]:
np.random.seed(3)
l0 = np.random.randint(0, 2, (samples, features))
l0  # input layer (features)

In [ ]:
np.linalg.matrix_rank(l0)

In [ ]:
y = np.random.randint(0, 2, (samples, 1))   # labels
y

In [ ]:
model = snn(a=0.01, act='sigmoid')

In [ ]:
model.fit(l0, y, 3000)

In [ ]:
model.predict(l0)

In [ ]:
model.predict(l0).round() == y

### Refactored Class

In [ ]:
class snn:
    def __init__(self, act='linear', a=0.01, steps=100,
                 verbose=False, psteps=200):
        self.act= act
        self.a = a
        self.steps = steps
        self.verbose = verbose
        self.psteps = psteps
    def forward(self):
        self.l2 = activation(np.dot(self.l0, self.w), self.act)
    def backward(self):
        self.e = self.l2 - self.y  # error ...
        d = self.e * activation(self.l2, self.act, True)  # ... is propagated ..
        u = self.a * np.dot(self.l0.T, d)  # ... backwards
        # u = a * np.dot(l0.T, e * activation(l2, self.act, True))
        self.w -= u
    def metrics(self, s):
        mse = (self.e ** 2).mean()
        acc = float(sum(self.l2.round() == self.y)[0] / len(self.y))
        self.res = pd.concat((self.res,
            pd.DataFrame({'mse': mse, 'acc': acc}, index=[s,]))
        )
        if s % self.psteps == 0 and self.verbose:
                print(f'step={s:5d} | mse={mse:.5f}')
                print(f'           | acc={acc:.5f}')
    def fit(self, l0, y, steps=None):
        self.l0 = l0
        self.y = y
        if steps is None:
            steps = self.steps
        self.res = pd.DataFrame()
        samples, features = l0.shape
        self.w = np.random.random((features, 1))
        for s in range(1, steps + 1):
            self.forward()
            self.backward()
            self.metrics(s)
    def predict(self, X):
        return activation(np.dot(X, self.w), self.act)

In [ ]:
model = snn(a=0.05, act='sigmoid')

In [ ]:
model.fit(l0, y, 1000)

In [ ]:
model.res.plot(figsize=(10, 6), secondary_y='mse');

## Learning &mdash; One Hidden Layer

Shallow neural network = ONE hidden layer = not DEEP neural network.

In [ ]:
class onn:
    def __init__(self, units=12, act='linear', a=0.01, steps=100,
                 verbose=False, psteps=200):
        self.units = units
        self.act= act
        self.a = a
        self.steps = steps
        self.verbose = verbose
        self.psteps = psteps
    def initialize(self):
        samples, features = self.l0.shape
        self.w0 = np.random.random((features, self.units))
        self.w1 = np.random.random((self.units, 1))
    def forward(self):
        self.l1 = activation(np.dot(self.l0, self.w0), self.act)
        self.l2 = activation(np.dot(self.l1, self.w1), self.act)
    def backward(self):
        self.e = self.l2 - self.y
        d2 = self.e * activation(self.l2, self.act, True)
        u2 = self.a * np.dot(self.l1.T, d2)
        self.w1 -= u2
        e1 = np.dot(d2, self.w1.T)
        d1 = e1 * activation(self.l1, self.act, True)
        u1 = self.a * np.dot(self.l0.T, d1)
        self.w0 -= u1
    def metrics(self, s):
        mse = (self.e ** 2).mean()
        acc = float(sum(self.l2.round() == self.y)[0] / len(self.y))
        self.res = pd.concat((self.res,
            pd.DataFrame({'mse': mse, 'acc': acc}, index=[s,]))
        )
        if s % self.psteps == 0 and self.verbose:
                print(f'step={s:5d} | mse={mse:.5f}')
                print(f'           | acc={acc:.5f}')
    def fit(self, l0, y, steps=None):
        self.l0 = l0
        self.y = y
        if steps is None:
            steps = self.steps
        self.res = pd.DataFrame()
        self.initialize()
        self.forward()
        for s in range(1, steps + 1):
            self.backward()
            self.forward()
            self.metrics(s)
    def predict(self, X):
        l1 = activation(np.dot(X, self.w0), self.act)
        l2 = activation(np.dot(l1, self.w1), self.act)
        return l2

### Estimation

In [ ]:
features = 5
samples = 10

In [ ]:
l0 = np.random.random((samples, features))

In [ ]:
np.linalg.matrix_rank(l0)

In [ ]:
y = np.random.random((samples, 1))

In [ ]:
model = onn(a=0.001, act='softplus')

In [ ]:
%time model.fit(l0, y, 5000)

In [ ]:
model.res.mse.min()

In [ ]:
model.res['mse'].iloc[40:].plot(figsize=(10, 6));

In [ ]:
res = pd.DataFrame({'y': y.flatten(),
                    'pred': model.predict(l0).flatten()},
                   index=range(len(y)))

In [ ]:
res.plot(kind='bar', figsize=(10, 6));

### Classification

In [ ]:
model = onn(a=0.025, act='sigmoid', steps=10000, verbose=True, psteps=2000)

In [ ]:
l0.round()

In [ ]:
y.round()

In [ ]:
model.fit(l0.round(), y.round())

In [ ]:
ax = model.res.plot(figsize=(10, 6), secondary_y='mse')
ax.get_legend().set_bbox_to_anchor((0.1, 0.5));

## Improved Optimizer

### Class

In [ ]:
class oonn:
    def __init__(self, units=12, act='linear', a=0.01, momentum=0.1,
                 steps=100, verbose=False, psteps=200, seed=None):
        self.units = units
        self.act= act
        self.a = a
        self.momentum = momentum
        self.steps = steps
        self.verbose = verbose
        self.psteps = psteps
        self.seed = seed
    def initialize(self):
        if self.seed is not None:
            np.random.seed(self.seed)
        samples, features = self.l0.shape
        self.w0 = np.random.random((features, self.units))
        self.w1 = np.random.random((self.units, 1))
        self.v1 = 0.
        self.v2 = 0.
        self.max_acc = 0.
        self.min_mse = 1000.
        self.best_w0 = self.w0.copy()
        self.best_w1 = self.w1.copy()
    def forward(self):
        self.l1 = activation(np.dot(self.l0, self.w0), self.act)
        self.l2 = activation(np.dot(self.l1, self.w1), self.act)
        self.e = self.l2 - self.y
    def backward(self):
        self.e = self.l2 - self.y
        d2 = self.e * activation(self.l2, self.act, True)
        v2 = self.v2 * self.momentum + self.a * np.dot(self.l1.T, d2)
        self.w1 = self.w1 + self.momentum * v2 - self.a * np.dot(self.l1.T, d2)
        self.v2 = v2
        e1 = np.dot(d2, self.w1.T)
        d1 = e1 * activation(self.l1, self.act, True)
        v1 = self.v1 * self.momentum + self.a * np.dot(self.l0.T, d1)
        self.w0 = self.w0 + self.momentum * v1 - self.a * np.dot(self.l0.T, d1)
        self.v1 = v1
    def metrics(self, s):
        mse = (self.e ** 2).mean()
        acc = float(sum(self.l2.round() == self.y)[0] / len(self.y))
        if acc > self.max_acc and self.target == 'acc':
            self.max_acc = acc
            self.best_w0 = self.w0.copy()
            self.best_w1 = self.w1.copy()
        elif mse < self.min_mse and self.target == 'mse':
            self.min_mse = mse
            self.best_w0 = self.w0.copy()
            self.best_w1 = self.w1.copy()
        opt = float(sum(self.predict(self.l0).round()[0] == self.y)[0] / len(self.y))
        self.res = pd.concat((self.res,
            pd.DataFrame({'mse': mse, 'acc': acc, 'opt': opt}, index=[s,]))
        )
        if s % self.psteps == 0 and self.verbose:
                print(f'step={s:5d} | mse={mse:.5f}')
                print(f'           | acc={acc:.5f}')
                print(f'           | opt={opt:.5f}')
    def fit(self, l0, y, target='acc', steps=None):
        self.target = target
        self.l0 = l0
        self.y = y
        if steps is None:
            steps = self.steps
        self.res = pd.DataFrame()
        self.initialize()
        self.forward()
        for s in range(1, steps + 1):
            self.backward()
            self.forward()
            self.metrics(s)
    def predict(self, X):
        l1 = activation(np.dot(X, self.best_w0), self.act)
        l2 = activation(np.dot(l1, self.best_w1), self.act)
        return l2
    def plot_metrics(self, ms=5, lw=2.0):
        model.res.plot(figsize=(10, 6), style=['o', 'o', '--'], ms=ms, lw=lw)

### Estimation

In [ ]:
features = 5
samples = 15

In [ ]:
np.random.seed(10)
l0 = np.random.random((samples, features))
l0[:5]  # input layer (features)

In [ ]:
y = np.random.random((samples, 1))   # labels
y[:5]

In [ ]:
model = oonn(units=64, a=0.0005, act='softplus', steps=1000,
            verbose=True, psteps=200)

In [ ]:
%time model.fit(l0, y, target='mse')

In [ ]:
model.min_mse

In [ ]:
model.plot_metrics(7, 2)

In [ ]:
res = pd.DataFrame({'y': y.flatten(),
                    'pred': model.predict(l0).flatten()},
                   index=range(len(y)))

In [ ]:
res.plot(kind='bar', figsize=(10, 6));

### Classification

#### Data

In [ ]:
l0 = l0.round()
l0[:5]

In [ ]:
y = y.round()
y[:5]

#### Simple Optimizer

In [ ]:
model = onn(a=0.05, act='sigmoid',
           verbose=True, psteps=1000)

In [ ]:
model.fit(l0, y, 4000)

In [ ]:
model.res.plot(figsize=(10, 6));

#### Improved Optimizer

In [ ]:
model = oonn(a=0.05, act='sigmoid',
             verbose=True, psteps=1000, seed=100)

In [ ]:
model.fit(l0, y, target='acc', steps=4000)

In [ ]:
model.plot_metrics(1.5, 2)

## Financial Data

### The Data

In [ ]:
url = 'http://hilpisch.com/tr_eikon_eod_data.csv'

In [ ]:
raw = pd.read_csv(url, index_col=0, parse_dates=True).dropna()

In [ ]:
raw.info()

In [ ]:
sym = 'AAPL.O'

In [ ]:
lags = 5
cols = []
data = pd.DataFrame(raw[sym])
data['r'] = np.log(data / data.shift(1))
data['d'] = np.where(data['r'] > 0, 1, 0)
for lag in range(1, lags+1):
    col = f'lag_{lag}'
    data[col] = data['d'].shift(lag)
    cols.append(col)
data.dropna(inplace=True)
data[cols] = data[cols].astype(int)
data = data.iloc[-500:]

In [ ]:
data.head()

### Fitting

In [ ]:
model = oonn(units=16 * 4, a=0.0001, momentum=0.05, act='sigmoid',
             verbose=True, psteps=100)

In [ ]:
y = data['d'].values.reshape(-1, 1)

In [ ]:
%time model.fit(data[cols].values, y, target='acc', steps=500)

In [ ]:
model.max_acc

In [ ]:
model.plot_metrics(3, 2)

## Batching

### Class

In [ ]:
class bonn(oonn):
    def forward(self):
        self.l1 = activation(np.dot(self.l0, self.w0), self.act)
        self.l2 = activation(np.dot(self.l1, self.w1), self.act)
        self.l1_ = activation(np.dot(self.l0_, self.w0), self.act)
        self.l2_ = activation(np.dot(self.l1_, self.w1), self.act)
    def metrics(self, s):
        self.e = self.l2_ - self.y_
        mse = (self.e ** 2).mean()
        acc = float(sum(self.l2_.round() == self.y_)[0] / len(self.y_))
        if acc > self.max_acc and self.target == 'acc':
            self.max_acc = acc
            self.best_w0 = self.w0.copy()
            self.best_w1 = self.w1.copy()
        elif mse < self.min_mse and self.target == 'mse':
            self.min_mse = mse
            self.best_w0 = self.w0.copy()
            self.best_w1 = self.w1.copy()
        opt = float(sum(self.predict(self.l0_).round() == self.y_)[0] / len(self.y_))
        self.res = pd.concat((self.res,
            pd.DataFrame({'mse': mse, 'acc': acc, 'opt': opt}, index=[s,]))
        )
        if s % self.psteps == 0 and self.verbose:
                print(f'step={s:5d} | mse={mse:.5f}')
                print(f'           | acc={acc:.5f}')
                print(f'           | opt={opt:.5f}')
    def fit(self, l0, y, target='acc', batch_size=32, steps=None):
        self.target = target
        self.l0_ = l0
        self.y_ = y
        if steps is None:
            steps = self.steps
        self.res = pd.DataFrame()
        n = int(len(y) / batch_size)
        i = 0
        self.min_mse = 0
        for b in range(n):
            self.l0 = self.l0_[b:b + batch_size]
            self.y = self.y_[b:b + batch_size]
            if b == 0:
                self.initialize()
            self.forward()
            for s in range(steps):
                i += 1
                self.backward()
                self.forward()
                if i % 5 == 0:
                    self.metrics(i)

### Fitting

In [ ]:
model = bonn(units=256, a=0.01, momentum=0.1,
             act='sigmoid', verbose=True, psteps=500, seed=10)

In [ ]:
y = data['d'].values.reshape(-1, 1)
len(y)

In [ ]:
%%time
model.fit(data[cols].values, y, target='acc',
                batch_size=32, steps=200)

In [ ]:
model.max_acc

In [ ]:
model.plot_metrics()

## Adjusting Features

### Bucketing

#### The Data

In [ ]:
lags = 5
data = pd.DataFrame(raw[sym])
data['r'] = np.log(data / data.shift(1))
data['d'] = np.where(data['r'] > 0, 1, 0)

In [ ]:
cols = []
for lag in range(1, lags+1):
    col = f'lag_{lag}'
    data[col] = data['r'].shift(lag)
    cols.append(col)
data.dropna(inplace=True)
data = data.iloc[-500:]

In [ ]:
data.head()

In [ ]:
mu = data['r'].mean()
std = data['r'].std()
bins = [mu - std / 2, mu, mu + std / 2]
# bins = [mu - std, mu - std / 2, mu, mu + std / 2, mu + std]

In [ ]:
l0 = np.digitize(data[cols].values, bins)
y = data['d'].values.reshape(-1, 1)

In [ ]:
l0

#### Fitting

In [ ]:
model = oonn(units=64, a=0.001, momentum=0.1,
             act='sigmoid', verbose=True, psteps=500, seed=100)

In [ ]:
%time model.fit(l0, y, target='acc', steps=2000)

In [ ]:
model.max_acc

In [ ]:
model.plot_metrics()

### Normalization

In [ ]:
model = oonn(units=128 * 8, a=0.005, momentum=0.1,
             act='sigmoid', verbose=True, psteps=500, seed=100)

In [ ]:
def normalize(x):
    return (x - x.mean()) / x.std()
def unit(x):
    return (x - x.min()) / (x.max() - x.min())

In [ ]:
# l0 = data[cols].values
# l0 = normalize(data[cols].values)
l0 = unit(data[cols].values)

In [ ]:
l0.min(), l0.max()

In [ ]:
l0[:5]

In [ ]:
y[:5]

In [ ]:
%time model.fit(l0, y, target='acc', steps=1000)

In [ ]:
model.max_acc

In [ ]:
model.plot_metrics()

## Validation

In [ ]:
class vonn(oonn):
    def metrics(self, s):
        mse = (self.e ** 2).mean()
        acc = float(sum(self.l2.round() == self.y)[0] / len(self.y))
        if acc > self.max_acc and self.target == 'acc':
            self.max_acc = acc
            self.best_w0 = self.w0.copy()
            self.best_w1 = self.w1.copy()
        elif mse < self.min_mse and self.target == 'mse':
            self.min_mse = mse
            self.best_w0 = self.w0.copy()
            self.best_w1 = self.w1.copy()
        opt = float(sum(self.predict(self.l0).round() == self.y)[0] / len(self.y))
        val = float(sum(self.predict(self.vl0).round() == self.vy)[0] / len(self.vy))
        self.res = pd.concat((self.res,
            pd.DataFrame({'mse': mse, 'acc': acc, 'opt': opt, 'val': val}, index=[s,]))
        )
        if s % self.psteps == 0 and self.verbose:
                print(f'step={s:5d} | mse={mse:.5f}')
                print(f'           | acc={acc:.5f}')
                print(f'           | opt={opt:.5f}')
                print(f'           | val={val:.5f}')
    def fit(self, l0, y, val=0.2, target='acc', steps=None):
        split = int(len(y) * (1 - val))
        self.l0 = l0[:split]
        self.y = y[:split]
        self.vl0 = l0[split:]
        self.vy = y[split:]
        self.target = target
        if steps is None:
            steps = self.steps
        self.res = pd.DataFrame()
        self.initialize()
        self.min_mse = 100.
        self.forward()
        for s in range(1, steps + 1):
            self.backward()
            self.forward()
            self.metrics(s)
    def plot_metrics(self, ms=5, lw=2.0):
        model.res.plot(figsize=(10, 6), style=['o', 'o', '--', 'o'], ms=ms, lw=lw)

In [ ]:
model = vonn(units=64, a=0.001, momentum=0.1,
             act='sigmoid', verbose=True, psteps=200, seed=10)

In [ ]:
l0 = normalize(data[cols].values)

In [ ]:
%time model.fit(l0, y, target='acc', val=0.20, steps=800)

In [ ]:
model.max_acc

In [ ]:
model.res.tail()

In [ ]:
model.plot_metrics(2)

## Vectorized Backtesting

### The Data

In [ ]:
lags = 10
sym = 'GLD'
data = pd.DataFrame(raw[sym])
data['r'] = np.log(data / data.shift(1))
data['d'] = np.where(data['r'] > 0, 1, 0)

In [ ]:
cols = []
for lag in range(1, lags+1):
    col = f'lag_{lag}'
    data[col] = data['r'].shift(lag)
    cols.append(col)
data.dropna(inplace=True)

In [ ]:
train = data.iloc[-1500:-500]

In [ ]:
test = data.iloc[-500:].copy()

### Fitting

In [ ]:
model = vonn(units=128, a=0.001, momentum=0.1,
             act='sigmoid', verbose=True, psteps=200, seed=10)

In [ ]:
l0 = normalize(train[cols].values)
y = train['d'].values.reshape(-1, 1)

In [ ]:
%time model.fit(l0, y, target='acc', steps=1000, val=0.20)

In [ ]:
model.max_acc

In [ ]:
model.res.tail()

In [ ]:
model.plot_metrics(2)

### Backtesting

In [ ]:
l0_ = normalize(test[cols].values)
y_ = test['d'].values.reshape(-1, 1)

In [ ]:
test['pred'] = model.predict(l0_).round()

In [ ]:
sum(test['pred'] == y_.flatten()) / len(y_)

In [ ]:
test['p'] = np.where(test['pred'] == 1, 1, -1)

In [ ]:
test['p'].value_counts()

In [ ]:
sum(test['p'].diff() != 0)

In [ ]:
test['s'] = test['p'] * test['r']

In [ ]:
test[['r', 's']].sum().apply(np.exp)

In [ ]:
test[['r', 's']].cumsum().apply(np.exp).plot(figsize=(10, 6));

## Universal Approximation with OONN

In [ ]:
x = np.linspace(0, 5 * np.pi, 100)
y = np.cos(x)

In [ ]:
i = 30
N = 20

In [ ]:
# x_ = normalize(np.random.random((N, N)))
x_ = unit(x[i:i+N]).reshape(-1, 1)
y_ = unit(y[i:i+N]).reshape(-1, 1)

In [ ]:
x_.shape, y_.shape

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(x_, y_, 'ro');

In [ ]:
model = oonn(units=512 * 4, a=0.01, momentum=0.1, steps=10000,
             act='sigmoid', verbose=True, psteps=2000, seed=1)

In [ ]:
%time model.fit(x_, y_, target='mse')

In [ ]:
model.plot_metrics()

In [ ]:
model.min_mse

In [ ]:
pred = model.predict(x_)

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(y_.flatten(), 'ro', label='original data')
plt.plot(pred, label='prediction')
plt.legend();

<img src='http://hilpisch.com/taim_logo.png' width="350px" align="right">